<a href="https://colab.research.google.com/github/rameshavinash94/CMPE257_NLP/blob/main/Week_9b_NLP_Week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTING LIBRARIES**

In [322]:
import spacy
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from numba import jit
import numpy as np

**SUPRESS WARNING**

In [122]:
#just to supress warnings
import warnings
warnings.filterwarnings("ignore")

**INSTALL SPACY ENGLISH LANGUAGE MODEL**

In [ ]:
#install spacy large english language model to use
! python -m spacy download en_core_web_lg
! pip install transformers

In [ ]:
exit()

**POEM CLASS WITH ALL FUNCTIONALITY**

In [323]:
class Poem:
  def __init__(self):
       self.nlp = spacy.load('en_core_web_lg')
       self.doc1_pos={'PROPN':set(), 'NUM':set(),'VERB':set(),'NOUN':set(),'ADJ':set()}
       self.doc2_pos={'PROPN':set(), 'NUM':set(),'VERB':set(),'NOUN':set(),'ADJ':set()}
       self.doc1_mappings={}
       self.doc2_mappings={}
       self.poet1_details={}
       self.poet2_details={}
    
  def UserInputbyPoet(self,poemsperpoet=10,no_of_poets=2):
        '''
        This Function is to get user query for the poet name and list of poems to scrap.
        '''
        self.poet1 = input('Enter the poet name to scrap {n} poems of him:'.format(n=poemsperpoet))
        self.poet2 = input('Enter the poet name to scrap {n} poems of him:'.format(n=poemsperpoet))
        self.poemsperpoet = poemsperpoet
        
        if not (self.poet1 or self.poet2):
          raise Exception('User input is empty or wrong please retype properly')
        
        #user input preprocessing per api
        self.poet1= self.poet1.title()
        self.poet2= self.poet2.title()
        
        return self.poet1,self.poet2,self.poemsperpoet
    
  def Scrapperbypoet(self,poemsperpoet=10):
    poet1={'poet':[],'title':[],'poem':[]}
    poet2={'poet':[],'title':[],'poem':[]}
    cookies = {
        'z': '1',
        'jt': '1',
        'tok': 'WzYwNTMxNzksImQ3ODVlZTdjMjk4YjQzODY5OWIyYmI1NTRiZjYxYWQzIl0%3D--62c210b76358a3e254c5f0219b490fd852676901',
        '_N': 'OWIwK3d4VUV0ZWQrZk1sZWxBWHV3TGIzQ210NDhkaWRWTXFPUlB4bldMY1BIVkMrT2dRZ2dnajBseEJ3NzRxZjRGZ2c0a2tXSVhEdThPZzZKOE9UK2pJanFRV094N2R2K1lUajB6MzE0MUcwWWJhbUNLTUpDZ2toVEd2dFo3OUtMZHFxYUpmb0ZsOG1aUzFpTjYyQy8zU0h4a04yQTBGUVZiY1gyRGwxR253ZUtIalg4Sk5xZnZNYTE2a2pEQjgxQk9aKytldHhreThUMmlLbE01MWRyUGttUWp3MXBxc0doZkhuVUdwcURBcVNRRmRSSmljVkJOMi8vSjRnUWJqRGNwaEhzYURJZjRhM1hOOEhGdG4yS0FNOTA5aXA3MHVjMVpMZmJCZ1RaVTFhZTNRQmhjWHZmdTJucUprczlZZmlURHcwVjdIREZLZm5JVm4rekljNm4vQVRlem50Yy9SM2R5ckNCYzZwUzViT1F2R0UvSjNiRnlaQ1F1QS9iZEtKRzNDcFpHWE0za3FZdzgyQjd0WFBoZ2FuTUZ6V1hidE8rUFpvd0htWTR2bzY3elNLZ3NqVUhVcWJxV25uRUt5NHFCS2ptSWlpQm9uUnRtUWZoK0dYMVJDQnc4cERKaCtkY29lM0JadGhXYU9rQ1Fvdkt4UVdaM1RqWmMyVzFqYlJhRnloZ0pHYlVsL2cwZm5uOEZXV2JLQ1JWODNrM3FZc3FDL3haWWl2cVpBPS0tQzQzVGljYVJtY3hIdUNpeFYrRmk3UT09--1af7abe737f99eccc42280051a1aaf758b3cc69a',
    }
    headers = {
        'authority': 'allpoetry.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-language': 'en-US,en;q=0.9',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'z=1; jt=1; tok=WzYwNTMxNzksImQ3ODVlZTdjMjk4YjQzODY5OWIyYmI1NTRiZjYxYWQzIl0%3D--62c210b76358a3e254c5f0219b490fd852676901; _N=OWIwK3d4VUV0ZWQrZk1sZWxBWHV3TGIzQ210NDhkaWRWTXFPUlB4bldMY1BIVkMrT2dRZ2dnajBseEJ3NzRxZjRGZ2c0a2tXSVhEdThPZzZKOE9UK2pJanFRV094N2R2K1lUajB6MzE0MUcwWWJhbUNLTUpDZ2toVEd2dFo3OUtMZHFxYUpmb0ZsOG1aUzFpTjYyQy8zU0h4a04yQTBGUVZiY1gyRGwxR253ZUtIalg4Sk5xZnZNYTE2a2pEQjgxQk9aKytldHhreThUMmlLbE01MWRyUGttUWp3MXBxc0doZkhuVUdwcURBcVNRRmRSSmljVkJOMi8vSjRnUWJqRGNwaEhzYURJZjRhM1hOOEhGdG4yS0FNOTA5aXA3MHVjMVpMZmJCZ1RaVTFhZTNRQmhjWHZmdTJucUprczlZZmlURHcwVjdIREZLZm5JVm4rekljNm4vQVRlem50Yy9SM2R5ckNCYzZwUzViT1F2R0UvSjNiRnlaQ1F1QS9iZEtKRzNDcFpHWE0za3FZdzgyQjd0WFBoZ2FuTUZ6V1hidE8rUFpvd0htWTR2bzY3elNLZ3NqVUhVcWJxV25uRUt5NHFCS2ptSWlpQm9uUnRtUWZoK0dYMVJDQnc4cERKaCtkY29lM0JadGhXYU9rQ1Fvdkt4UVdaM1RqWmMyVzFqYlJhRnloZ0pHYlVsL2cwZm5uOEZXV2JLQ1JWODNrM3FZc3FDL3haWWl2cVpBPS0tQzQzVGljYVJtY3hIdUNpeFYrRmk3UT09--1af7abe737f99eccc42280051a1aaf758b3cc69a',
        'if-none-match': 'W/"6ebecc9a316dba275701ba119fa8ae21"',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'sec-gpc': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36',
    }

    poet1_poems = requests.get('https://allpoetry.com/classics/alpha/{poet1}'.format(poet1=self.poet1), headers=headers, cookies=cookies)
    poet2_poems = requests.get('https://allpoetry.com/classics/alpha/{poet2}'.format(poet2=self.poet2), headers=headers, cookies=cookies)

    
    doc1=BeautifulSoup(poet1_poems.text,'html.parser')
    doc2=BeautifulSoup(poet2_poems.text,'html.parser')
    
    poet1_details={'poet':[],'title':[],'poem':[],'poem_url':[]}
    poet2_details={'poet':[],'title':[],'poem':[],'poem_url':[]}

    for ul in doc1.find_all('ul',attrs={"class" : "gapped my2"}):
      ahref = ul.find_all("a",href=True)[:poemsperpoet]
      for x in ahref:
        tmp=self.Scrapperbypoem(x['href'])
        poet1_details['poet'].append(tmp['poet'])
        poet1_details['title'].append(x.text)
        poet1_details['poem'].append(tmp['poem'])
        poet1_details['poem_url'].append('https://allpoetry.com{url}'.format(url=x['href']))

    for ul in doc2.find_all('ul',attrs={"class" : "gapped my2"}):
      ahref = ul.find_all("a",href=True)[:poemsperpoet]
      for x in ahref:
        tmp=self.Scrapperbypoem(x['href'])
        poet2_details['poet'].append(tmp['poet'])
        poet2_details['title'].append(x.text)
        poet2_details['poem'].append(tmp['poem'])
        poet2_details['poem_url'].append('https://allpoetry.com{url}'.format(url=x['href']))

    self.poet1_details=poet1_details
    self.poet2_details=poet2_details
    return self.poet1_details,self.poet2_details

  def Scrapperbypoem(self,poem):
      '''
        #Iterate and Scrap data to get poem and author name and store in Df
      '''
      poem_info={'poet':'','poem':''}
      url='https://allpoetry.com{iter}'.format(iter=poem)
      html_content = requests.get(url).text
      doc=BeautifulSoup(html_content,'html.parser')

      for div in doc.find_all('div',attrs={"class" : re.compile("^bio*")})[0]:
          ahref = div.find_all("a",attrs={"class": "u nocolor"})
          for x in ahref:
            if x!=None:
                  poem_info['poet']= x.text

      for div in doc.find_all('div',attrs={"class" : re.compile("^orig")}):
          poem_info['poem']= div.text
      
      return poem_info

  def Convert_df(self):
    '''
    This function converts the peom info to DF.
    '''
    self.poet_1_details_df = pd.DataFrame(self.poet1_details)
    self.poet_2_details_df = pd.DataFrame(self.poet2_details)

  def lemmatization(self,corpus):
    '''
    This Function removes stop words and lemmatizes the provided corpus
    :return: Stop word removed & Canonical form corpus (canonical)
    '''
    doc=self.nlp(corpus)
    temp=[]
    for token in doc:
        if not token.is_stop:
            temp.append(token.lemma_)

    canonical = " ".join(temp)

    return canonical
  
  def remove_sp(self,corpus):
    '''
    this function is used to remove special characters from the corpus
    '''
    import re
    return re.sub(r"[^a-zA-Z0-9]+", ' ', corpus)
  
  def PreprocessPoems(self,lematization=True):
    '''
    This Function is to preprocess user input,calls lemmatization function and extract only the nceesary POS if requested.
    '''
    if lematization==True:
      self.poet_1_details_df['lemma_stop_word_removal'] = self.poet_1_details_df['poem'].map(lambda x : self.lemmatization(x))
      self.poet_2_details_df['lemma_stop_word_removal'] = self.poet_2_details_df['poem'].map(lambda x : self.lemmatization(x))
      self.poet_1_details_df['cleaned'] = self.poet_1_details_df['lemma_stop_word_removal'].map(lambda x: self.remove_sp(x))
      self.poet_2_details_df['cleaned'] = self.poet_2_details_df['lemma_stop_word_removal'].map(lambda x: self.remove_sp(x))
    else:
      self.poet_1_details_df['cleaned'] = self.poet_1_details_df['poem'].map(lambda x: self.remove_sp(x))
      self.poet_2_details_df['cleaned'] = self.poet_2_details_df['poem'].map(lambda x: self.remove_sp(x))

  def find_Pos(self):
    '''
    this function is used to find pos
    '''
    for x in self.poet_1_details_df['cleaned']:
      doc1=self.nlp(x)
      for token in doc1:
        if token.pos_ == 'PROPN':
          self.doc1_pos['PROPN'].add(token.text)
        
        elif token.pos_ == 'NUM':
          self.doc1_pos['NUM'].add(token.text)
        
        elif token.pos_ == 'VERB':
          self.doc1_pos['VERB'].add(token.text)
        
        elif token.pos_ == 'NOUN':
          self.doc1_pos['NOUN'].add(token.text)
        
        elif token.pos_ == 'ADJ':
          self.doc1_pos['ADJ'].add(token.text)
  
    for x in self.poet_2_details_df['cleaned']:
      doc2=self.nlp(x)
      for token in doc2:
        if token.pos_ == 'PROPN':
          self.doc2_pos['PROPN'].add(token.text)
        
        elif token.pos_ == 'NUM':
          self.doc2_pos['NUM'].add(token.text)
        
        elif token.pos_ == 'VERB':
          self.doc2_pos['VERB'].add(token.text)
        
        elif token.pos_ == 'NOUN':
          self.doc2_pos['NOUN'].add(token.text)
        
        elif token.pos_ == 'ADJ':
          self.doc2_pos['ADJ'].add(token.text)
    
    return self.doc1_pos,self.doc2_pos

  def loopover(self,set1,set2):
    '''
    just loop over 2 sets and find cosine similarity
    '''
    @jit(nopython=True)
    def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
      assert(u.shape[0] == v.shape[0])
      uv = 0
      uu = 0
      vv = 0
      for i in range(u.shape[0]):
          uv += u[i]*v[i]
          uu += u[i]*u[i]
          vv += v[i]*v[i]
      cos_theta = 1
      if uu!=0 and vv!=0:
          cos_theta = uv/np.sqrt(uu*vv)
      return cos_theta

    tmp1=set1
    tmp2=set2
    doc_mappings={}
    for x in tmp1:
      w1=self.nlp(x)
      max=-1
      for y in tmp2:
        w2=self.nlp(y)
        simliariy_score=cosine_similarity_numba(w1.vector,w2.vector)
        if simliariy_score > 0.5:
          doc_mappings[x]=[y,simliariy_score]
          #max=simliariy_score
          break
        else:
          doc_mappings[x]=[x,None]
    
    return doc_mappings

  def POS_ComputeSimilarity(self):
    '''
    This function computes cosine similarity
    '''
    for x in self.doc1_pos:
        self.doc1_mappings.update(self.loopover(self.doc1_pos[x],self.doc2_pos[x]))
    
    for x in self.doc2_pos:
        self.doc2_mappings.update(self.loopover(self.doc2_pos[x],self.doc1_pos[x]))

  def Create_new_Poems(self):
    '''
    This functino creates new poems based on the provided one's.
    '''
    self.new_poem1_poem2=[]
    self.new_poem2_poem1=[]
    for x in self.poet_1_details_df['lemma_stop_word_removal']:
      new_poem3=str(x)
      for key, value in self.doc1_mappings.items():
      # Replace key POS with value POS in string
        new_poem3 = new_poem3.replace(str(key), str(value[0]))
      self.new_poem1_poem2.append(str(new_poem3))
      
    for x in self.poet_2_details_df['lemma_stop_word_removal']:
      new_poem4=str(x)
      for key, value in self.doc2_mappings.items():
      # Replace key POS with value POS in string
        new_poem4 = new_poem4.replace(str(key), str(value[0]))
      
      self.new_poem2_poem1.append(str(new_poem4))
    
    self.poet_1_details_df['new_poem'] = self.new_poem1_poem2
    self.poet_2_details_df['new_poem'] = self.new_poem2_poem1

**CREATE OBJECT FOR THE CLASS**

In [324]:
#create object for the class
tmp = Poem()

**GET INPUT FOR 2 POEMS**

In [ ]:
# Poet1 - Edward Dyer   
# Poet2 - Walt Whitman

In [325]:
#GET USER INPUT
tmp.UserInputbyPoet()

Enter the poet name to scrap 10 poems of him:Edward Dyer
Enter the poet name to scrap 10 poems of him:Walt Whitman


('Edward Dyer', 'Walt Whitman', 10)

**RUN SCRAPPER TO EXTRACT POEM FROM ALLPOETRY**

In [326]:
#Call scrapper function to scrap poems from both poets
tmp.Scrapperbypoet()

({'poem': ["Hee that his mirth hath loste,\n \xa0Whose comfort is dismaid,\nWhose hope is vaine, whose faith is scorned,\n \xa0Whose trust is all betraid,\n\n\n \xa0If he have held them deare,\n \xa0And cannot cease to moane,\nCome, let him take his place by me;\n \xa0He shall not rue alone.\n\n\n \xa0But if the smalest sweete\n \xa0Be mixt with all his sowre;\nIf in the day, the moneth, the yeare,\n \xa0He finde one lightsome hower,\n\n\n \xa0Then rest he by himself;\n \xa0He is noe mate for me,\nWhose hope is falen, whose succor voyde,\n \xa0Whose hart his death must be.\n\n\n \xa0Yet not the wishèd death,\n \xa0That hathe noe plainte nor lacke,\nWhich, making free the better parte,\n \xa0Is onely nature's sacke.\n\n\n \xa0Oh me! that wer too well,\n \xa0My death is of the minde,\nWhich alwayes yeeldès extreame paines,\n \xa0Yet keepes the worst behind.\n\n\n \xa0As one that lives in shewe\n \xa0But inwardly doth die,\nWhose knowledge is a bloody field\n \xa0Wheare all hope slaine do

**CONVERT THE INFO TO DF TO EASIER VISUALIZATION**

In [327]:
#convert to df both poem1, poem2
tmp.Convert_df()

In [328]:
#view df1
tmp.poet_1_details_df

,poet,title,poem,poem_url
0,Edward Dyer,A Fancy,"Hee that his mirth hath loste,\n Whose comfor...",https://allpoetry.com/A-Fancy
1,Edward Dyer,A Lady Forsaken Complayneth,"If pleasures be in painfulness, in pleasures d...",https://allpoetry.com/A-Lady-Forsaken-Complayneth
2,Edward Dyer,A Silent Love,"\r\n\tThe lowest trees have tops, the ant her ...",https://allpoetry.com/The-lowest-trees-have-tops
3,Edward Dyer,Coridon to his Phillis,"Alas my hart, mine eye hath wrongèd thee,\nPre...",https://allpoetry.com/Coridon-to-his-Phillis
4,Edward Dyer,I would and I would not,I woulde it were not as it is\nOr that I cared...,https://allpoetry.com/I-would-and-I-would-not
5,Edward Dyer,Love-Contradictions,"As rare to heare as seldome to be seene,\nIt c...",https://allpoetry.com/Love-Contradictions
6,Edward Dyer,Love-Despondency,Devyde my tymes and rate my wretched howres\r\...,https://allpoetry.com/Love-Despondency
7,Edward Dyer,My Mind to me a Kingdom is,My mind to me a kingdom is;\n Such perf...,https://allpoetry.com/My-Mind-to-me-a-Kingdom-is
8,Edward Dyer,The Faire Amarillis,Amarillis was full fayre:\nThe goodlyest mayde...,https://allpoetry.com/The-Faire-Amarillis
9,Edward Dyer,The man of Woe,The mann whose thoughtes agaynste him do consp...,https://allpoetry.com/The-man-of-Woe


In [329]:
#view df2
tmp.poet_2_details_df

,poet,title,poem,poem_url
0,Walt Whitman,1861,Arm'd year! year of the struggle!\nNo dainty r...,https://allpoetry.com/poem/8451663-1861-by-Wal...
1,Walt Whitman,A Boston Ballad,"TO get betimes in Boston town, I rose this mor...",https://allpoetry.com/A-Boston-Ballad
2,Walt Whitman,A Broadway Pageant,"Over the western sea, hither from Niphon come,...",https://allpoetry.com/A-Broadway-Pageant
3,Walt Whitman,A Carol Of Harvest For 1867,A SONG of the good green grass!\n A song no ...,https://allpoetry.com/A-Carol-Of-Harvest-For-1867
4,Walt Whitman,"A child said, What is the grass?","A child said, What is the grass? fetching it t...","https://allpoetry.com/A-child-said,-What-is-th..."
5,Walt Whitman,A Child's Amaze,"SILENT and amazed, even when a little boy,\nI ...",https://allpoetry.com/A-Child's-Amaze
6,Walt Whitman,A Clear Midnight,"This is thy hour O Soul, thy free flight into ...",https://allpoetry.com/A-Clear-Midnight
7,Walt Whitman,A Farm-Picture,THROUGH the ample open door of the peaceful co...,https://allpoetry.com/A-Farm-Picture
8,Walt Whitman,A Glimpse,"A GLIMPSE, through an interstice caught,\nOf a...",https://allpoetry.com/A-Glimpse
9,Walt Whitman,A Hand-Mirror,Hold it up sternly! See this it sends back! (W...,https://allpoetry.com/A-Hand-Mirror


**PERFORM NLP PREPROCESSING - STOP WORD REMOVAL/LEMMATIZATION/CLEAN THE CORPUS**

In [330]:
#run preporcessing function, for now have set lemma to false
tmp.PreprocessPoems()

In [331]:
tmp.poet_1_details_df

,poet,title,poem,poem_url,lemma_stop_word_removal,cleaned
0,Edward Dyer,A Fancy,"Hee that his mirth hath loste,\n Whose comfor...",https://allpoetry.com/A-Fancy,"hee mirth hath loste , \n comfort dismaid , ...",hee mirth hath loste comfort dismaid hope vain...
1,Edward Dyer,A Lady Forsaken Complayneth,"If pleasures be in painfulness, in pleasures d...",https://allpoetry.com/A-Lady-Forsaken-Complayneth,"pleasure painfulness , pleasure doth body rest...",pleasure painfulness pleasure doth body rest j...
2,Edward Dyer,A Silent Love,"\r\n\tThe lowest trees have tops, the ant her ...",https://allpoetry.com/The-lowest-trees-have-tops,"\r\n\t low tree top , ant gall , \r\n\t fly sp...",low tree top ant gall fly spleen little spark...
3,Edward Dyer,Coridon to his Phillis,"Alas my hart, mine eye hath wrongèd thee,\nPre...",https://allpoetry.com/Coridon-to-his-Phillis,"alas hart , eye hath wrongèd thee , \n Presump...",alas hart eye hath wrong d thee Presumptious e...
4,Edward Dyer,I would and I would not,I woulde it were not as it is\nOr that I cared...,https://allpoetry.com/I-would-and-I-would-not,woulde \n care yea ; \n woulde thoughte amiss ...,woulde care yea woulde thoughte amiss amiss mi...
5,Edward Dyer,Love-Contradictions,"As rare to heare as seldome to be seene,\nIt c...",https://allpoetry.com/Love-Contradictions,"rare heare seldome seene , \n hathe bene \n fi...",rare heare seldome seene hathe bene fire burne...
6,Edward Dyer,Love-Despondency,Devyde my tymes and rate my wretched howres\r\...,https://allpoetry.com/Love-Despondency,Devyde tyme rate wretched howre \r\n day month...,Devyde tyme rate wretched howre day month fro ...
7,Edward Dyer,My Mind to me a Kingdom is,My mind to me a kingdom is;\n Such perf...,https://allpoetry.com/My-Mind-to-me-a-Kingdom-is,mind kingdom ; \n perfect joy find \n ...,mind kingdom perfect joy find excel bliss God ...
8,Edward Dyer,The Faire Amarillis,Amarillis was full fayre:\nThe goodlyest mayde...,https://allpoetry.com/The-Faire-Amarillis,Amarillis fayre : \n goodlyest mayde \n east u...,Amarillis fayre goodlyest mayde east unto west...
9,Edward Dyer,The man of Woe,The mann whose thoughtes agaynste him do consp...,https://allpoetry.com/The-man-of-Woe,"mann thoughtes agaynste conspyre , \n Mishap...",mann thoughtes agaynste conspyre Mishapp story...


In [332]:
tmp.poet_2_details_df

,poet,title,poem,poem_url,lemma_stop_word_removal,cleaned
0,Walt Whitman,1861,Arm'd year! year of the struggle!\nNo dainty r...,https://allpoetry.com/poem/8451663-1861-by-Wal...,Arm'd year ! year struggle ! \n dainty rhyme s...,Arm d year year struggle dainty rhyme sentimen...
1,Walt Whitman,A Boston Ballad,"TO get betimes in Boston town, I rose this mor...",https://allpoetry.com/A-Boston-Ballad,"betimes Boston town , rise morning early ; \n ...",betimes Boston town rise morning early good pl...
2,Walt Whitman,A Broadway Pageant,"Over the western sea, hither from Niphon come,...",https://allpoetry.com/A-Broadway-Pageant,"western sea , hither Niphon come , \n courteou...",western sea hither Niphon come courteous swart...
3,Walt Whitman,A Carol Of Harvest For 1867,A SONG of the good green grass!\n A song no ...,https://allpoetry.com/A-Carol-Of-Harvest-For-1867,song good green grass ! \n song city street...,song good green grass song city street song fa...
4,Walt Whitman,"A child said, What is the grass?","A child said, What is the grass? fetching it t...","https://allpoetry.com/A-child-said,-What-is-th...","child say , grass ? fetch \n hand ; \n answer ...",child say grass fetch hand answer child I know...
5,Walt Whitman,A Child's Amaze,"SILENT and amazed, even when a little boy,\nI ...",https://allpoetry.com/A-Child's-Amaze,"silent amazed , little boy , \n remember hear ...",silent amazed little boy remember hear preache...
6,Walt Whitman,A Clear Midnight,"This is thy hour O Soul, thy free flight into ...",https://allpoetry.com/A-Clear-Midnight,"thy hour o soul , thy free flight wordless , \...",thy hour o soul thy free flight wordless away ...
7,Walt Whitman,A Farm-Picture,THROUGH the ample open door of the peaceful co...,https://allpoetry.com/A-Farm-Picture,"ample open door peaceful country barn , \n sun...",ample open door peaceful country barn sun ligh...
8,Walt Whitman,A Glimpse,"A GLIMPSE, through an interstice caught,\nOf a...",https://allpoetry.com/A-Glimpse,"glimpse , interstice catch , \n crowd workman ...",glimpse interstice catch crowd workman driver ...
9,Walt Whitman,A Hand-Mirror,Hold it up sternly! See this it sends back! (W...,https://allpoetry.com/A-Hand-Mirror,hold sternly ! send ! ( ? ? ) \n outside fair ...,hold sternly send outside fair costume ashe fi...


**FIND POS**

In [333]:
print(tmp.find_Pos(), end=' ')

({'PROPN': {'o', 'owle', 'stealle', 'professe', 'tymes', 'seeme', 'coulde', 'yeald', 'Aspis', 'Althoughe', 'courteseye', 'darlynge', 'place', 'rue', 'heer', 'disdaind', 'Whoe', 'winge', 'anie', 'pyttye', 's', 'cocke', 'palme', 'corne', 'bondman', 'thou', 'sylence', 'Butt', 'fate', 'west', 'healpe', 'loathe', 'sigh', 'Infortune', 'felowe', 'force', 'scoff', 'leaue', 'meane', 'regarde', 'stringe', 'myghte', 'eve', 'blamles', 'fawn', 'chaire', 'Springe', 'Cittie', 'rocke', 'Doth', 'happ', 'thine', 'wean', 'Mishap', 'suffyse', 'yore', 'reservd', 'mayste', 'Fame', 'peace', 'muste', 'cipre', 'sustaine', 'bondage', 'soughte', 'leafe', 'lande', 'vaine', 'withall', 'Howe', 'power', 'woode', 'drerye', 'equall', 'thoughtes', 'woemen', 'Caramell', 'turnde', 'quyte', 'sweet', 'Thrise', 'aloofe', 'Sunne', 'nothynge', 'appeere', 'knowe', 'comfort', 'water', 'mynds', 'Pytheas', 'Love', 'flowre', 'state', 'sounde', 'alas', 'Lackinge', 'daye', 'strong', 'booke', 'wranglynge', 'flockes', 'mistris', 'yeal

**COMPUTE SIMILARITY OF POS**

In [334]:
tmp.POS_ComputeSimilarity()

In [335]:
tmp.doc1_mappings

{'o': ['o', 1.0],
 'owle': ['o', 1.0],
 'stealle': ['o', 1.0],
 'professe': ['o', 1.0],
 'tymes': ['seiz', 1.0],
 'seeme': ['seiz', 1.0],
 'coulde': ['o', 1.0],
 'yeald': ['o', 1.0],
 'Aspis': ['seiz', 1.0],
 'Althoughe': ['o', 1.0],
 'courteseye': ['o', 1.0],
 'darlynge': ['o', 1.0],
 'place': ['place', 1.0],
 'rue': ['debouch', 1.0],
 'heer': ['seiz', 1.0],
 'disdaind': ['o', 1.0],
 'Whoe': ['o', 1.0],
 'winge': ['seiz', 1.0],
 'anie': ['o', 1.0],
 'pyttye': ['o', 1.0],
 's': ['seiz', 1.0],
 'cocke': ['seiz', 1.0],
 'palme': ['seiz', 1.0],
 'corne': ['seiz', 1.0],
 'bondman': ['o', 1.0],
 'thou': ['thou', 1.0],
 'sylence': ['o', 1.0],
 'Butt': ['seiz', 1.0],
 'fate': ['seiz', 1.0],
 'west': ['South', 0.9348259424355018],
 'healpe': ['o', 1.0],
 'loathe': ['debouch', 1.0],
 'sigh': ['seiz', 1.0],
 'Infortune': ['o', 1.0],
 'felowe': ['o', 1.0],
 'force': ['roarer', 1.0],
 'scoff': ['seiz', 1.0],
 'leaue': ['o', 1.0],
 'meane': ['o', 1.0],
 'regarde': ['seiz', 1.0],
 'stringe': ['o', 1

In [336]:
tmp.doc2_mappings

{'o': ['o', 1.0],
 'Ancient': ['owle', 1.0],
 'panorama': ['owle', 1.0],
 'South': ['foregotten', 1.0],
 'Hebrews': ['owle', 1.0],
 'western': ['foregotten', 1.0],
 'sea': ['pytty', 1.0],
 'port': ['owle', 1.0],
 'elate': ['owle', 1.0],
 'group': ['owle', 1.0],
 'hill': ['hill', 1.0],
 'machine': ['pytty', 1.0],
 'Illinois': ['owle', 1.0],
 'bellied': ['owle', 1.0],
 'interlace': ['owle', 1.0],
 'seiz': ['o', 1.0],
 'place': ['pytty', 1.0],
 'ade': ['owle', 1.0],
 'move': ['pytty', 1.0],
 'Dispensatress': ['o', 1.0],
 'Sea': ['owle', 1.0],
 'Harden': ['owle', 1.0],
 'kaleidoscope': ['owle', 1.0],
 'Busy': ['owle', 1.0],
 'Man': ['owle', 1.0],
 'fit': ['foregotten', 1.0],
 'workman': ['pytty', 1.0],
 'Parliament': ['owle', 1.0],
 'street': ['pytty', 1.0],
 'Tennessee': ['owle', 1.0],
 'Dame': ['owle', 1.0],
 'window': ['pytty', 1.0],
 'Missouri': ['owle', 1.0],
 'demi': ['owle', 1.0],
 'new': ['foregotten', 1.0],
 'thou': ['enioye', 1.0],
 'pasture': ['owle', 1.0],
 'Assyria': ['owle', 

In [337]:
tmp.poet_1_details_df

,poet,title,poem,poem_url,lemma_stop_word_removal,cleaned
0,Edward Dyer,A Fancy,"Hee that his mirth hath loste,\n Whose comfor...",https://allpoetry.com/A-Fancy,"hee mirth hath loste , \n comfort dismaid , ...",hee mirth hath loste comfort dismaid hope vain...
1,Edward Dyer,A Lady Forsaken Complayneth,"If pleasures be in painfulness, in pleasures d...",https://allpoetry.com/A-Lady-Forsaken-Complayneth,"pleasure painfulness , pleasure doth body rest...",pleasure painfulness pleasure doth body rest j...
2,Edward Dyer,A Silent Love,"\r\n\tThe lowest trees have tops, the ant her ...",https://allpoetry.com/The-lowest-trees-have-tops,"\r\n\t low tree top , ant gall , \r\n\t fly sp...",low tree top ant gall fly spleen little spark...
3,Edward Dyer,Coridon to his Phillis,"Alas my hart, mine eye hath wrongèd thee,\nPre...",https://allpoetry.com/Coridon-to-his-Phillis,"alas hart , eye hath wrongèd thee , \n Presump...",alas hart eye hath wrong d thee Presumptious e...
4,Edward Dyer,I would and I would not,I woulde it were not as it is\nOr that I cared...,https://allpoetry.com/I-would-and-I-would-not,woulde \n care yea ; \n woulde thoughte amiss ...,woulde care yea woulde thoughte amiss amiss mi...
5,Edward Dyer,Love-Contradictions,"As rare to heare as seldome to be seene,\nIt c...",https://allpoetry.com/Love-Contradictions,"rare heare seldome seene , \n hathe bene \n fi...",rare heare seldome seene hathe bene fire burne...
6,Edward Dyer,Love-Despondency,Devyde my tymes and rate my wretched howres\r\...,https://allpoetry.com/Love-Despondency,Devyde tyme rate wretched howre \r\n day month...,Devyde tyme rate wretched howre day month fro ...
7,Edward Dyer,My Mind to me a Kingdom is,My mind to me a kingdom is;\n Such perf...,https://allpoetry.com/My-Mind-to-me-a-Kingdom-is,mind kingdom ; \n perfect joy find \n ...,mind kingdom perfect joy find excel bliss God ...
8,Edward Dyer,The Faire Amarillis,Amarillis was full fayre:\nThe goodlyest mayde...,https://allpoetry.com/The-Faire-Amarillis,Amarillis fayre : \n goodlyest mayde \n east u...,Amarillis fayre goodlyest mayde east unto west...
9,Edward Dyer,The man of Woe,The mann whose thoughtes agaynste him do consp...,https://allpoetry.com/The-man-of-Woe,"mann thoughtes agaynste conspyre , \n Mishap...",mann thoughtes agaynste conspyre Mishapp story...


**CREATE NEW POEMS**

In [338]:
#create new poems
tmp.Create_new_Poems()

In [339]:
tmp.doc1_mappings

{'o': ['o', 1.0],
 'owle': ['o', 1.0],
 'stealle': ['o', 1.0],
 'professe': ['o', 1.0],
 'tymes': ['seiz', 1.0],
 'seeme': ['seiz', 1.0],
 'coulde': ['o', 1.0],
 'yeald': ['o', 1.0],
 'Aspis': ['seiz', 1.0],
 'Althoughe': ['o', 1.0],
 'courteseye': ['o', 1.0],
 'darlynge': ['o', 1.0],
 'place': ['place', 1.0],
 'rue': ['debouch', 1.0],
 'heer': ['seiz', 1.0],
 'disdaind': ['o', 1.0],
 'Whoe': ['o', 1.0],
 'winge': ['seiz', 1.0],
 'anie': ['o', 1.0],
 'pyttye': ['o', 1.0],
 's': ['seiz', 1.0],
 'cocke': ['seiz', 1.0],
 'palme': ['seiz', 1.0],
 'corne': ['seiz', 1.0],
 'bondman': ['o', 1.0],
 'thou': ['thou', 1.0],
 'sylence': ['o', 1.0],
 'Butt': ['seiz', 1.0],
 'fate': ['seiz', 1.0],
 'west': ['South', 0.9348259424355018],
 'healpe': ['o', 1.0],
 'loathe': ['debouch', 1.0],
 'sigh': ['seiz', 1.0],
 'Infortune': ['o', 1.0],
 'felowe': ['o', 1.0],
 'force': ['roarer', 1.0],
 'scoff': ['seiz', 1.0],
 'leaue': ['o', 1.0],
 'meane': ['o', 1.0],
 'regarde': ['seiz', 1.0],
 'stringe': ['o', 1

In [340]:
for x in tmp.poet_1_details_df['new_poem']:
  print(str(x))

hee roarer thou looizte , 
   roadebouchr roadebouchrioizmairoadebouchr , 
 place o , roadebouchr oizcorn , 
   troizeizt betrairoadebouchr , 


   holroadebouchr roadebouchro , 
   croadebouchrize moaseiz , 
 take , take place ; 
   oizhall roadebouchdebouchbouch . 


   oizmaleoizt oizweete 
   o oizowdebouch ; 
 roadebouchray , o , o , 
   finroadebouchdebouch paleoizome oiz , 


   debouchoizt ; 
   oiz roarer , 
 place o , oizuccor voyroadebouchdebouch , 
   oiz roadebouchrtakeh . 


   wioizhèroadebouchr roadebouchrtakeh , 
   thoue oiz oiz o , 
 , take fdebouche well oiz , 
   oseizly natudebouch oizacke . 


   oh ! wer , 
   roadebouchrtakeh minroadebouchdebouch , 
 o seizelroadebouchrèoiz oiz oizoiz , 
   take baroadebouchr . 


   louisianian oizhewe 
   inwarroadebouchrly roadebouchdebouchh roadebouchrie , 
 knoroadebouchrge blooroadebouchry fielroadebouchr 
   o place oizlaiseiz roadebouchdebouchh roarer ; 


   oize o , 
   oizpirit , oizacrifize 
 unto roadebouchr woiz a

In [341]:
for x in tmp.poet_2_details_df['new_poem']:
  print(str(x))

owle'foreeattten owlpytty ! owlpytty owleruggle ! 
 foreeatttenainty pytty senpyttyntal pytty pytty , terpyttyle owlpytty ! 
 foregotten o , pyttyt foreeatttenesk , pytty cowlenza 
          pytty ; 
 owlerong pytty , owlect , enioye foreeattten enioye , aforeeatttenvance , 
          enioye pytty shoulforeeatttener , 
 - griowlele boforeeattteny pyttyburnt pytty hanforeeattten — pytty 
          pytty , 
 feel enioye louforeeattten — pyttyorous pytty pytty 
          pytty ; 
 mascupytty pytty , o owlpytty , fall amiforeeattten foregotten pytty , 
 amiforeeattten pytty owlepyttytan feel , pytty , 
          foreeatttenweller owlepyttytan ; 
 foregotten owleep pytty pytty owle 
          Inforeeattteniana , 
 rapiforeeatttenly pytty Weowle sppyttyy pytty , foreeatttenescenforeeattten 
          o ;                                                 
 foregotten hill , owle , foreeatttepytty 
          owle hill ; 
 southpyttyforeeattten owle owle hill , 
          owle hill , 
 feel pytty

**LDA**

In [342]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [343]:
poet1=[]
poet2=[]
nlp=spacy.load('en_core_web_sm')

testing_df1 = tmp.poet_1_details_df
final1=[]
for x in testing_df1['cleaned']:
  test=[]
  for token in nlp(x):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        test.append(token.lemma_)
  final1.append(test)

testing_df2 = tmp.poet_2_details_df
final2=[]
for x in testing_df2['cleaned']:
  test=[]
  for token in nlp(x):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        test.append(token.lemma_)
  final2.append(test)

final3=[]
for x in testing_df1['new_poem']:
  test=[]
  for token in nlp(x):
    if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
      test.append(token.lemma_)
  final3.append(test)

final4=[]
for x in testing_df2['new_poem']:
  test=[]
  for token in nlp(x):
    if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
      test.append(token.lemma_)
  final4.append(test)

In [344]:
#poet 1 all poems
id2word1 = corpora.Dictionary(final1)
corpus1 = []
for text in final1:
    new = id2word1.doc2bow(text)
    corpus1.append(new)

lda_model1 = gensim.models.LdaMulticore(corpus1, num_topics=10, id2word=id2word1, passes=2, workers=2)

In [345]:
#poet 2 all poems

id2word2 = corpora.Dictionary(final2)
corpus2 = []
for text in final2:
    new = id2word2.doc2bow(text)
    corpus2.append(new)

lda_model2 = gensim.models.LdaMulticore(corpus2, num_topics=10, id2word=id2word2, passes=2, workers=2)

In [346]:
#poet1-poet2 all poems

id2word3 = corpora.Dictionary(final3)
corpus3 = []
for text in final3:
    new = id2word3.doc2bow(text)
    corpus3.append(new)

lda_model3 = gensim.models.LdaMulticore(corpus3, num_topics=10, id2word=id2word3, passes=2, workers=2)

In [347]:
#poet2-poet1 all poems

id2word4 = corpora.Dictionary(final4)
corpus4 = []
for text in final4:
    new = id2word4.doc2bow(text)
    corpus4.append(new)

lda_model4 = gensim.models.LdaMulticore(corpus4, num_topics=10, id2word=id2word4, passes=2, workers=2)

In [348]:
#print topics and words poet1
for idx, topic in lda_model1.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.017*"woulde" + 0.014*"hope" + 0.009*"man" + 0.009*"coulde" + 0.008*"happy" + 0.008*"happ" + 0.007*"thy" + 0.006*"care" + 0.006*"desyre" + 0.006*"present"
Topic: 1 
Words: 0.008*"theire" + 0.008*"Amarillis" + 0.008*"Butt" + 0.007*"d" + 0.007*"Coridon" + 0.006*"wth" + 0.006*"tyme" + 0.006*"thy" + 0.006*"eche" + 0.006*"lyfe"
Topic: 2 
Words: 0.003*"hope" + 0.003*"mind" + 0.003*"woulde" + 0.002*"care" + 0.002*"man" + 0.002*"come" + 0.002*"love" + 0.002*"d" + 0.002*"daye" + 0.002*"fall"
Topic: 3 
Words: 0.015*"love" + 0.014*"eye" + 0.009*"shall" + 0.009*"noe" + 0.009*"Phillis" + 0.009*"hath" + 0.008*"come" + 0.007*"die" + 0.007*"doth" + 0.007*"hope"
Topic: 4 
Words: 0.002*"love" + 0.002*"daye" + 0.002*"care" + 0.002*"hope" + 0.002*"thy" + 0.002*"eye" + 0.002*"wth" + 0.002*"tyme" + 0.002*"d" + 0.002*"shall"
Topic: 5 
Words: 0.016*"mind" + 0.016*"love" + 0.012*"d" + 0.009*"pleasure" + 0.008*"mean" + 0.008*"pain" + 0.008*"sore" + 0.007*"doth" + 0.006*"find" + 0.006*"hart"
To

In [349]:
#print topics and words poet1
final=set()
for index, topic in lda_model1.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet 1 important Topic words\n')
print(final)

Poet 1 important Topic words

{'o', 'case', 'coulde', 'mynde', 'fynde', 'Coridon', 'woulde', 'fall', 'word', 'Whoe', 'eche', 'pain', 'small', 'thou', 'Butt', 'tyme', 'nay', 'good', 'kynde', 'loathe', 'offence', 'nowe', 'ye', 'mind', 'meane', 'eve', 'thoughte', 'happ', 'loe', 'faith', 'suffyse', 'liberty', 'delyghte', 'hart', 'power', 'flame', 'come', 'Maye', 'Caramell', 'bear', 'beare', 'want', 'matter', 'know', 'thee', 'soe', 'Theire', 'knowe', 'ease', 'maye', 'comfort', 'yeeld', 'wishe', 'theyr', 'doth', 'selfe', 'flowre', 'state', 'accord', 'face', 'think', 'find', 'joye', 'daye', 'foe', 'touche', 'yealde', 'torment', 'mean', 'prayse', 'thy', 'woe', 'live', 'amiss', 'happy', 'die', 'life', 'finde', 'great', 'Phillis', 'king', 'mann', 'present', 'hath', 'sore', 'lyfe', 'stand', 'man', 'joy', 'despayre', 'fortune', 'thrall', 'hathe', 'heare', 'nyghte', 'paine', 'Amarillis', 'compare', 'leave', 'd', 'look', 'end', 'wthout', 'pleasure', 'cause', 'love', 'death', 'suche', 'Finis', 'shalb

In [350]:
#print topics and words poet2

for idx, topic in lda_model2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.017*"old" + 0.016*"man" + 0.014*"mother" + 0.013*"grass" + 0.013*"child" + 0.013*"guess" + 0.010*"young" + 0.010*"lap" + 0.010*"woman" + 0.010*"dark"
Topic: 1 
Words: 0.010*"old" + 0.009*"d" + 0.008*"man" + 0.008*"way" + 0.008*"Boston" + 0.007*"come" + 0.007*"love" + 0.006*"shall" + 0.006*"look" + 0.006*"phantom"
Topic: 2 
Words: 0.017*"d" + 0.013*"army" + 0.010*"field" + 0.009*"o" + 0.009*"thou" + 0.008*"harvest" + 0.008*"Heroes" + 0.008*"song" + 0.007*"grass" + 0.007*"land"
Topic: 3 
Words: 0.024*"year" + 0.019*"d" + 0.011*"voice" + 0.011*"see" + 0.011*"clothe" + 0.007*"great" + 0.007*"Manhattan" + 0.007*"river" + 0.007*"man" + 0.007*"blue"
Topic: 4 
Words: 0.018*"chant" + 0.017*"d" + 0.015*"Libertad" + 0.012*"come" + 0.010*"island" + 0.007*"foot" + 0.007*"day" + 0.007*"pageant" + 0.007*"city" + 0.007*"thousand"
Topic: 5 
Words: 0.011*"voice" + 0.010*"eye" + 0.010*"look" + 0.010*"step" + 0.006*"hand" + 0.005*"face" + 0.005*"flash" + 0.005*"send" + 0.005*"costume" +

In [351]:
#print topics and words poet2
final=set()
for index, topic in lda_model2.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet 2 important Topic words\n')
print(final)

Poet 2 important Topic words

{'o', 'take', 'zone', 'brain', 'move', 'sprout', 'toil', 'go', 'word', 'springy', 'silent', 'thou', 'heart', 'perceive', 'pasture', 'lap', 'cannon', 'good', 'hour', 'mother', 'behold', 'ye', 'clothe', 'voice', 'star', 'guess', 'phantom', 'ample', 'President', 'gather', 'chant', 'Manhattan', 'blue', 'foot', 'tree', 'step', 'river', 'cross', 'stream', 'grave', 'hay', 'day', 'open', 'maize', 'barn', 'Heroes', 'come', 'sing', 'know', 'thee', 'army', 'sour', 'weapon', 'callous', 'answer', 'vista', 'dark', 'gaze', 'city', 'harvest', 'seat', 'hint', 'think', 'face', 'procession', 'Thou', 'strong', 'hand', 'sex', 'cattle', 'long', 'woman', 'little', 'white', 'gait', 'Asia', 'hold', 'door', 'costume', 'flash', 'young', 'race', 'corner', 'life', 'great', 'roof', 'old', 'room', 'West', 'field', 'green', 'blood', 'thousand', 'soldier', 'government', 'country', 'haze', 'box', 'see', 'man', 'send', 'begin', 'gun', 'sad', 'Libertad', 'horizon', 'leave', 'd', 'look', 'off

In [352]:
#print topics and words poet1-poet2

for idx, topic in lda_model3.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.010*"oiz" + 0.008*"seiz" + 0.007*"o" + 0.006*"roadebouchr" + 0.005*"take" + 0.003*"louisianian" + 0.003*"oeiz" + 0.003*"oizeiz" + 0.003*"roarer" + 0.003*"taket"
Topic: 1 
Words: 0.002*"oiz" + 0.002*"o" + 0.002*"roadebouchr" + 0.002*"seiz" + 0.002*"oizeiz" + 0.002*"oize" + 0.002*"louisianian" + 0.002*"take" + 0.002*"mynroadebouchdebouch" + 0.002*"fynroadebouchdebouch"
Topic: 2 
Words: 0.074*"roadebouchr" + 0.063*"oiz" + 0.056*"o" + 0.032*"roarer" + 0.026*"take" + 0.024*"seiz" + 0.024*"taket" + 0.020*"place" + 0.020*"debouch" + 0.020*"thou"
Topic: 3 
Words: 0.007*"oiz" + 0.006*"o" + 0.005*"roadebouchr" + 0.004*"take" + 0.003*"place" + 0.003*"seiz" + 0.003*"roarer" + 0.003*"thou" + 0.003*"debouch" + 0.003*"oizeiz"
Topic: 4 
Words: 0.090*"o" + 0.079*"oiz" + 0.045*"roadebouchr" + 0.043*"seiz" + 0.025*"take" + 0.017*"oizeiz" + 0.014*"place" + 0.014*"louisianian" + 0.014*"oeiz" + 0.013*"roarer"
Topic: 5 
Words: 0.021*"o" + 0.019*"oiz" + 0.014*"seiz" + 0.011*"roadebouchr" + 

In [353]:
#print topics and words
final=set()
for index, topic in lda_model3.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet1-poet2 important Topic words\n')
print(final)

Poet1-poet2 important Topic words

{'o', 'unnatural', 'take', 'seizèroadebouchr', 'countless', 'roadebouchdebouchh', 'troizeizt', 'roadebouchrtakeh', 'oizy', 'seiz', 'place', 'toil', 'prayoize', 'roadebouchdebouchoizpyghte', 'ear', 'thou', 'debouch', 'nay', 'roadebouchr', 'roadebouchdebouchlyght', 'good', 'hour', 'fdebouche', 'beauty', 'oizeize', 'pale', 'oeiz', 'oizhall', 'oiztrong', 'roarer', 'roadebouchdebouchoizydebouch', 'binroadebouchr', 'oizweet', 'roadebouchdebouch', 'worroadebouchr', 'feilroadebouchr', 'torseiz', 'plroadebouchrizudebouch', 'finroadebouchr', 'Coriroadebouchron', 'roar', 'minroadebouchdebouch', 'debouchoizt', 'roadebouchdebouchbouch', 'enroadebouchr', 'louisianian', 'houlroadebouchr', 'roadebouchrseiz', 'roadebouchdebouchpale', 'oize', 'wearseiz', 'oizweete', 'roadebouchray', 'juroadebouchrge', 'oèroadebouchr', 'well', 'caoize', 'oizowdebouch', 'oiz', 'roadebouchrful', 'accorroadebouchr', 'roadebouchdeboucho', 'roadebouchrie', 'pdebouchoizent', 'taket', 'minroad

In [354]:
#print topics and words Poet2-poet1

for idx, topic in lda_model4.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.040*"pytty" + 0.028*"owle" + 0.015*"foregotten" + 0.010*"enioye" + 0.006*"foreeattten" + 0.006*"feel" + 0.005*"islanforeeattten" + 0.004*"enioowlesanforeeattten" + 0.004*"worlforeeattten" + 0.003*"hill"
Topic: 1 
Words: 0.314*"pytty" + 0.103*"owle" + 0.067*"foregotten" + 0.055*"enioye" + 0.031*"foreeattten" + 0.025*"feel" + 0.009*"hill" + 0.007*"eat" + 0.006*"o" + 0.006*"curse"
Topic: 2 
Words: 0.003*"pytty" + 0.003*"owle" + 0.003*"foregotten" + 0.003*"foreeattten" + 0.003*"feel" + 0.003*"quiet" + 0.003*"enioye" + 0.003*"apytty" + 0.003*"Sunpytty" + 0.003*"fowle"
Topic: 3 
Words: 0.004*"pytty" + 0.004*"owle" + 0.003*"foregotten" + 0.003*"enioye" + 0.003*"foreeattten" + 0.003*"feel" + 0.003*"curse" + 0.003*"fiowle" + 0.003*"harveowle" + 0.003*"pyttyg"
Topic: 4 
Words: 0.003*"pytty" + 0.003*"foregotten" + 0.003*"owle" + 0.003*"enioye" + 0.003*"feel" + 0.003*"foreeattten" + 0.003*"quiet" + 0.003*"apytty" + 0.003*"Sunpytty" + 0.003*"fowle"
Topic: 5 
Words: 0.003*"pytty" 

In [355]:
#print topics and words
final=set()
for index, topic in lda_model3.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet2-poet1 important Topic words\n')
print(final)

Poet2-poet1 important Topic words

{'o', 'unnatural', 'take', 'seizèroadebouchr', 'countless', 'roadebouchdebouchh', 'troizeizt', 'roadebouchrtakeh', 'oizy', 'seiz', 'place', 'toil', 'prayoize', 'roadebouchdebouchoizpyghte', 'ear', 'thou', 'debouch', 'nay', 'roadebouchr', 'roadebouchdebouchlyght', 'good', 'hour', 'fdebouche', 'beauty', 'oizeize', 'pale', 'oeiz', 'oizhall', 'oiztrong', 'roarer', 'roadebouchdebouchoizydebouch', 'binroadebouchr', 'oizweet', 'roadebouchdebouch', 'worroadebouchr', 'feilroadebouchr', 'torseiz', 'plroadebouchrizudebouch', 'finroadebouchr', 'Coriroadebouchron', 'roar', 'minroadebouchdebouch', 'debouchoizt', 'roadebouchdebouchbouch', 'enroadebouchr', 'louisianian', 'houlroadebouchr', 'roadebouchrseiz', 'roadebouchdebouchpale', 'oize', 'wearseiz', 'oizweete', 'roadebouchray', 'juroadebouchrge', 'oèroadebouchr', 'well', 'caoize', 'oizowdebouch', 'oiz', 'roadebouchrful', 'accorroadebouchr', 'roadebouchdeboucho', 'roadebouchrie', 'pdebouchoizent', 'taket', 'minroad

In [356]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model1, corpus1, id2word1)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.158199  0.001365       1        1  30.168235
1     -0.045169 -0.119489       2        1  24.307814
9     -0.119977  0.077897       3        1  20.130430
5      0.036184  0.076734       4        1  10.147143
0     -0.027700 -0.036817       5        1   9.820743
7     -0.003212  0.000516       6        1   5.210597
6     -0.007172  0.003464       7        1   0.053759
2      0.005513 -0.000872       8        1   0.053759
4      0.004048 -0.007196       9        1   0.053759
8     -0.000715  0.004398      10        1   0.053759, topic_info=      Term       Freq      Total Category  logprob  loglift
182   love  17.000000  17.000000  Default  30.0000  30.0000
156   hope  10.000000  10.000000  Default  29.0000  29.0000
675   daye   7.000000   7.000000  Default  28.0000  28.0000
69       d  11.000000  11.000000  Default  27.0000  27.0000
54    care   8.000000   8.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
156   hope   0.001274  10.761919  Topic10  -6.6566  -1.5135
395    eye   0.001272  12.329590  Topic10  -6.6578  -1.6506
57   cause   0.001213   4.892952  Topic10  -6.7054  -0.7740
289    thy   0.001215   8.154336  Topic10  -6.7039  -1.2833
249  shall   0.001210   7.819001  Topic10  -6.7077  -1.2451

[693 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
858       2  0.917917  Amarillis
859       2  0.502924  Amaryllis
861       2  0.498365        Att
643       3  0.856892    Besydes
863       2  0.581487       Butt
...     ...       ...        ...
325       5  0.380423        yea
640       3  0.703274     yealde
327       1  0.660594      yeeld
799       3  0.522763       yeer
386       4  1.048241      youth

[511 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 10, 6, 1, 8, 7, 3, 5, 9])

In [357]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, corpus2, id2word2)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.154480 -0.028439       1        1  41.053664
4     -0.140614 -0.075954       2        1  24.740492
1     -0.030845  0.101943       3        1  16.592047
0     -0.032510  0.072546       4        1   7.557511
3      0.016987 -0.026683       5        1   5.926296
5      0.015635 -0.027228       6        1   3.044754
6      0.014092 -0.000003       7        1   0.926103
9      0.006700  0.011019       8        1   0.053044
7     -0.006976 -0.010383       9        1   0.053044
8      0.003052 -0.016818      10        1   0.053044, topic_info=       Term       Freq      Total Category  logprob  loglift
24        d  25.000000  25.000000  Default  30.0000  30.0000
319   chant  10.000000  10.000000  Default  29.0000  29.0000
670   field   8.000000   8.000000  Default  28.0000  28.0000
585    army  10.000000  10.000000  Default  27.0000  27.0000
693   grass   7.000000   7.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
82    voice   0.001667   6.655619  Topic10  -6.3720  -0.7505
693   grass   0.001732   7.876570  Topic10  -6.3334  -0.8803
77   strong   0.001563   5.422352  Topic10  -6.4360  -0.6096
228    star   0.001521   5.251676  Topic10  -6.4638  -0.6053
190   march   0.001506   5.448731  Topic10  -6.4734  -0.6518

[659 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
258       1  0.768987  America
258       2  0.256329  America
261       2  0.716872     Asia
263       2  0.829951  Assyria
86        3  0.608904   Boston
...     ...       ...      ...
256       1  0.230018    young
256       2  0.230018    young
256       3  0.230018    young
256       4  0.230018    young
992       4  0.700901     zone

[368 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 1, 4, 6, 7, 10, 8, 9])

In [358]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model3, corpus3, id2word3)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.116008 -0.082333       1        1  52.835891
4     -0.119716  0.072914       2        1  46.686935
9     -0.019242  0.013273       3        1   0.059648
5     -0.006590  0.010726       4        1   0.059647
8      0.015884 -0.008861       5        1   0.059647
0      0.023555  0.006207       6        1   0.059647
3      0.034653 -0.008492       7        1   0.059646
7      0.059752 -0.003102       8        1   0.059646
1      0.062661  0.000204       9        1   0.059646
6      0.065051 -0.000536      10        1   0.059646, topic_info=                       Term        Freq       Total Category  logprob  loglift
84                        o  118.000000  118.000000  Default  30.0000  30.0000
85                      oiz  115.000000  115.000000  Default  29.0000  29.0000
147             roadebouchr   99.000000   99.000000  Default  28.0000  28.0000
162                    seiz   54.000000   54.000000  Default  27.0000  27.0000
171                    take   42.000000   42.000000  Default  26.0000  26.0000
..                      ...         ...         ...      ...      ...      ...
363   woulroadebouchdebouch    0.001535    7.810175  Topic10  -6.4692  -1.1101
261       Coriroadebouchron    0.001535    7.644384  Topic10  -6.4692  -1.0886
236               countless    0.001535    2.056174  Topic10  -6.4692   0.2245
237                     ear    0.001535    1.130438  Topic10  -6.4692   0.8227
143  roadebouchdebouchbouch    0.001535    9.412880  Topic10  -6.4692  -1.2968

[721 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
1         1  0.901164              Carthoize
261       1  0.523260      Coriroadebouchron
261       2  0.523260      Coriroadebouchron
300       2  0.868113          Deoizydebouch
377       2  0.909784  Devyroadebouchdebouch
...     ...       ...                    ...
196       1  0.897613               woroizte
198       1  1.050856         worroadebouchr
363       1  0.128038  woulroadebouchdebouch
363       2  0.896267  woulroadebouchdebouch
487       1  0.884612        yielroadebouchr

[350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 10, 6, 9, 1, 4, 8, 2, 7])

In [359]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model4, corpus4, id2word4)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.215837 -0.019500       1        1  92.951956
6     -0.200076  0.022419       2        1   6.610292
0     -0.016313 -0.006970       3        1   0.054720
7      0.046272  0.004666       4        1   0.054719
3      0.057152  0.001453       5        1   0.054719
2      0.065760 -0.000414       6        1   0.054719
4      0.065760 -0.000414       7        1   0.054719
5      0.065760 -0.000414       8        1   0.054719
8      0.065760 -0.000414       9        1   0.054719
9      0.065760 -0.000414      10        1   0.054719, topic_info=               Term        Freq       Total Category  logprob  loglift
31            pytty  553.000000  553.000000  Default  30.0000  30.0000
24             owle  180.000000  180.000000  Default  29.0000  29.0000
17       foregotten  118.000000  118.000000  Default  28.0000  28.0000
7            enioye   96.000000   96.000000  Default  27.0000  27.0000
10      foreeattten   54.000000   54.000000  Default  26.0000  26.0000
..              ...         ...         ...      ...      ...      ...
234          fiowle    0.002485    9.329226  Topic10  -5.9738  -0.7198
69   holforeeattten    0.002485    2.570056  Topic10  -5.9738   0.5694
330  worforeeattten    0.002485    1.163581  Topic10  -5.9738   1.3619
226           curse    0.002485   10.362745  Topic10  -5.9738  -0.8249
119        enioyery    0.002485    2.468576  Topic10  -5.9738   0.6097

[627 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
204       1  1.030118  Beholforeeattteneowle
2         1  0.822945      aforeeatttenvance
3         1  0.734415         amiforeeattten
210       1  1.009439                 apytty
4         1  0.939630                assault
...     ...       ...                    ...
200       1  0.906951         winforeeattten
328       1  1.037186                 wiowle
330       2  0.859416         worforeeattten
370       1  0.735337     worforeeatttenless
202       1  0.947888        worlforeeattten

[132 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 1, 8, 4, 3, 5, 6, 9, 10])

**GSDMM**

In [ ]:
#!pip install git+https://github.com/rwalk/gsdmm.git

In [360]:
from gsdmm import MovieGroupProcess

In [361]:
mgp = MovieGroupProcess(K=3, alpha=0.01, beta=0.01, n_iters=30)
# create variable containing length of dictionary/vocab
vocab = set(x for y in final1 for x in y)
n_terms = len(vocab)
n_docs = 10

In [362]:
model = mgp.fit(final1, n_terms)

In stage 0: transferred 8 clusters with 2 clusters populated
In stage 1: transferred 1 clusters with 3 clusters populated
In stage 2: transferred 0 clusters with 3 clusters populated
In stage 3: transferred 0 clusters with 3 clusters populated
In stage 4: transferred 0 clusters with 3 clusters populated
In stage 5: transferred 0 clusters with 3 clusters populated
In stage 6: transferred 0 clusters with 3 clusters populated
In stage 7: transferred 0 clusters with 3 clusters populated
In stage 8: transferred 0 clusters with 3 clusters populated
In stage 9: transferred 0 clusters with 3 clusters populated
In stage 10: transferred 0 clusters with 3 clusters populated
In stage 11: transferred 0 clusters with 3 clusters populated
In stage 12: transferred 0 clusters with 3 clusters populated
In stage 13: transferred 0 clusters with 3 clusters populated
In stage 14: transferred 0 clusters with 3 clusters populated
In stage 15: transferred 0 clusters with 3 clusters populated
In stage 16: trans

In [363]:
# print number of documents per topic
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('Most important clusters (by number of docs inside):', top_index)

# define function to get top words per topic
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster, sort_dicts))

# get top words in topics
top_words(mgp.cluster_word_distribution, top_index, 20)

Number of documents per topic : [1 1 8]
Most important clusters (by number of docs inside): [2 1 0]

Cluster 2 : [('love', 19), ('d', 15), ('eye', 14), ('hope', 14), ('care', 11), ('thy', 10), ('daye', 9), ('tyme', 9), ('shall', 9), ('doth', 9), ('man', 8), ('lyfe', 8), ('woulde', 8), ('Coridon', 8), ('come', 8), ('hath', 8), ('noe', 8), ('good', 7), ('wth', 7), ('theire', 7)]

Cluster 1 : [('love', 3), ('low', 1), ('tree', 1), ('top', 1), ('ant', 1), ('gall', 1), ('fly', 1), ('spleen', 1), ('little', 1), ('spark', 1), ('heat', 1), ('slender', 1), ('hair', 1), ('cast', 1), ('shadow', 1), ('small', 1), ('bee', 1), ('ste', 1), ('great', 1), ('Seas', 1)]

Cluster 0 : [('love', 4), ('hathe', 3), ('live', 2), ('rare', 1), ('heare', 1), ('seldome', 1), ('seene', 1), ('bene', 1), ('fire', 1), ('burne', 1), ('perfecte', 1), ('heate', 1), ('flame', 1), ('matter', 1), ('yealde', 1), ('straunger', 1), ('case', 1), ('true', 1), ('profe', 1), ('knowe', 1)]


**TEXT SUMMARIZATION**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

In [ ]:
poem1_poem2_summary=[]
for x in tmp.poet_1_details_df['new_poem']:
  ARTICLE_TO_SUMMARIZE = str(x)
  inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=512, return_tensors="pt")
  summary_ids = model.generate(inputs["input_ids"])
  poem1_poem2_summary.append(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

In [216]:
poem1_poem2_summary

['Hee that his slave loSouthe, VaSouth agess is sismais, VaSouth agess is Dispitchsatforshinesdebouchingat, Dispitchsatforshinessodebouching place is scorIllinoiss, VaSouth truSouth is all facingtra',
 'If pleasuforshines facing in heartfulIllinoisss, in pleasuforshines soth my bosy forshineSouth, If joDispitchsatforshinesss accors peaceh agessful Dispitchsatforshiness is in my shineeSouth:',
 'The loweSouth tforshinees have sit, the interstice her tears, The forget her heart, the Southern place his place; agessns slplaceebouchingr tears caSouth shasows, slavegh but Southern, agessns facinges have Southings,',
 'VaSouth Corison, the VaSouth Dispitchsatforshinessodebouching agess soth debouching flocks, VaSouth agess to sraw, but is not srawIllinois to agess flocks.',
 'The busss of agess aforshine Southarvdebouching peaceh fVast agessns SouthDig his thou countlesss his agess; My SouthDig, if agess the Dispitchsatforshiness debouchingbouchingt sr VaSouth rDis',
 'agess Southraunger cade

In [215]:
poem2_poem1_summary=[]
for x in tmp.poet_2_details_df['new_poem']:
  ARTICLE_TO_SUMMARIZE = str(x)
  inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=512,return_tensors="pt")
  summary_ids = model.generate(inputs["input_ids"])
  poem2_poem1_summary.append(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

In [217]:
poem2_poem1_summary

["owle'kyndtake owlcourteseye!",
 "Tcourtescourteseye getake betakeimecourtakeeseowle in Bocourtakeeseowtakeakeon takequiet, I rocourtakeeseowlee takehicourtakeeseowle courtescourteseye courteseyely; Here'scourtakeeseowle a take",
 "owle courtakeeseowle courtescourteseye'kyndtake courtescourteseyene-courtakeeseowlecourteseyeyndtakeekyndtake envoycourtakeeseowle errankyndtake.",
 'A ScourtescourteseyeNG of takehe takeakeokyndtake small gracourtakeeseowlecourtakeeseowle!',
 'A courtescourteseyer I guecourtakeeseowlecourtakeeseowle itake icourtakeeseowle courtescourteseye ankyndtake owleakendtakeecourtakeowleignekyndtakelyndtake, A courteseowleignekynd',
 'Sunkyndtakeay ankyndtake amazekyndtake, even when a profescourteseyeketakele courtescourteseye, I take I hcourteseyekyndtake takehe courtescourteseye every Sunkyndtakeay takeake owle in hicourtake',
 'All images are copyrighted.',
 'A courtakeeseowleun-profescourteseyeke pacourtakeeseowtakeakeure fiowle, witakeh cataketakele ankyndtake 

**WRITE_TO_JSON**

In [220]:
poet1_details=tmp.poet1_details
poet1_details['new_poem'] = tmp.new_poem1_poem2
poet1_details['POS']=tmp.doc1_pos
poet1_details['summary']=poem1_poem2_summary

poet2_details=tmp.poet2_details
poet2_details['new_poem'] = tmp.new_poem1_poem2
poet2_details['POS']=tmp.doc2_pos
poet2_details['summary'] = poem2_poem1_summary

In [ ]:
with open("poet1.json", "w") as outfile:
  json.dump(poet1_details, outfile)

with open("poet2.json", "w") as outfile:
  json.dump(poet2_details, outfile)

**GET POET DETAILS FROM POET WIKI PAGE INSTEAD NEWS ARTICLES**



In [226]:
! pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=82655018a5566b9a471ffa9ee70c2f86cdd912ee8da8298ec6b279394fc6b818
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [251]:
import wikipedia

In [252]:
def Retrive(top_n=1):
  '''
  This Function returns Wikipedia Pages results based on user query
  '''
  search_results_pages=[]
  for x in range(2):
    search_text = input('Enter Poet Name')
    search_text = search_text.strip()
    search_results = wikipedia.search(search_text, results=top_n)
    for docs in search_results:
        search_results_pages.append(wikipedia.page(docs,
                auto_suggest=False))
  return search_results_pages

In [253]:
pages = Retrive()

Enter Poet Namewalt whitman
Enter Poet Nameedward dyer


In [254]:
poet_1 = pages[0].content

In [255]:
poet_2 = pages[1].content

In [256]:
def lemma_pos(corpus):
  preprocesed  = re.sub(r"[^a-zA-Z0-9]+", ' ', corpus)
  nlp=spacy.load('en_core_web_sm')
  lemmatized = []
  for token in nlp(preprocesed):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        lemmatized.append(token.lemma_)
  return lemmatized

In [257]:
poet_1_lemma_corpus = lemma_pos(poet_1)
poet_2_lemma_corpus = lemma_pos(poet_2)

In [317]:
print(poet_1_lemma_corpus, end='')

['Walter', 'Whitman', 'May', '31', '1819', 'March', '26', '1892', 'american', 'poet', 'essayist', 'journalist', 'humanist', 'part', 'transition', 'transcendentalism', 'realism', 'incorporate', 'view', 'work', 'Whitman', 'influential', 'poet', 'american', 'canon', 'call', 'father', 'free', 'verse', 'work', 'controversial', 'time', '1855', 'poetry', 'collection', 'leave', 'Grass', 'describe', 'obscene', 'overt', 'sensuality', 'Whitman', 'own', 'life', 'come', 'scrutiny', 'presume', 'homosexuality', 'bear', 'Huntington', 'Long', 'Island', 'child', 'much', 'career', 'reside', 'Brooklyn', 'age', '11', 'leave', 'formal', 'schooling', 'go', 'work', 'Whitman', 'work', 'journalist', 'teacher', 'government', 'clerk', 'Whitman', 'major', 'poetry', 'collection', 'leave', 'Grass', 'publish', '1855', 'own', 'money', 'become', 'know', 'work', 'attempt', 'reach', 'common', 'person', 'american', 'epic', 'continue', 'expand', 'revise', 'death', '1892', 'American', 'Civil', 'War', 'go', 'Washington', 'D'

In [318]:
print(poet_2_lemma_corpus, end='')

['Sir', 'Edward', 'Dyer', 'October', '1543', 'May', '1607', 'english', 'courtier', 'poet', 'Life', 'son', 'Sir', 'Thomas', 'Dyer', 'Kt', 'bear', 'Sharpham', 'Park', 'Glastonbury', 'Somerset', 'educate', 'accord', 'Anthony', 'Wood', 'Balliol', 'College', 'Oxford', 'Broadgates', 'Hall', 'Pembroke', 'College', 'Oxford', 'leave', 'take', 'degree', 'time', 'appear', 'Elizabeth', 's', 'court', 'first', 'patron', 'Robert', 'Dudley', 'Earl', 'Leicester', 'seem', 'think', 'put', 'rival', 'Sir', 'Christopher', 'Hatton', 'queen', 's', 'favour', 'mention', 'Gabriel', 'Harvey', 'Sir', 'Philip', 'Sidney', 'one', 'ornament', 'court', 'Sidney', 'will', 'bequeathed', 'book', 'Fulke', 'Greville', 'Dyer', 'make', 'steward', 'Woodstock', '1570', 'employ', 'Elizabeth', 'mission', '1584', 'Low', 'Countries', '1589', 'send', 'Denmark', 'commission', 'inquire', 'manor', 'alienate', 'crown', 'west', 'country', 'please', 'queen', 'receive', 'grant', 'forfeit', 'land', 'Somerset', '1588', 'return', 'Member', 'Pa

**POET1**

In [273]:
final_corpus1=[]
id2wordd1 = corpora.Dictionary([poet_1_lemma_corpus])
for text in [poet_1_lemma_corpus]:
    new = id2wordd1.doc2bow(text)
    final_corpus1.append(new)

In [284]:
lda_modelpoet1 = gensim.models.LdaMulticore(final_corpus1, num_topics=2, id2word=id2wordd1, passes=2, workers=2)

In [285]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_modelpoet1, final_corpus1, id2wordd1)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.013446  0.0       1        1  99.985971
0     -0.013446  0.0       2        1   0.014029, topic_info=         Term        Freq       Total Category  logprob  loglift
753   Whitman  180.000000  180.000000  Default  30.0000  30.0000
739      Walt   43.000000   43.000000  Default  29.0000  29.0000
2055    write   30.000000   30.000000  Default  28.0000  28.0000
541       New   22.000000   22.000000  Default  27.0000  27.0000
358     Grass   27.000000   27.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
150   America    0.000797   14.081407   Topic2  -6.6546  -0.9072
937      call    0.000757   15.030020   Topic2  -6.7060  -1.0238
1591     part    0.000690   10.814406   Topic2  -6.7993  -0.7879
1630   poetry    0.000757   15.901156   Topic2  -6.7070  -1.0810
2051    would    0.000686   10.821449   Topic2  -6.8045  -0.7937

[132 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
65        1  0.832295      1948
88        1  0.837397      2013
150       1  0.994219   America
177       1  0.832157     Bacon
211       1  0.833017   Britten
...     ...       ...       ...
2004      1  0.835388  veracity
2047      1  1.012287      work
2051      1  1.016500     would
2055      1  1.008091     write
2062      1  1.030369      year

[77 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [311]:
for index, topic in lda_modelpoet1.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

Topic: 9 
Words: ['Whitman', 'Walt', 'write', 'New', 'Grass', 'poet', 'work', 'leave', 'Leaves', 'York', 'time', 'edition', 'american', 'year', 'include', 'one', 'publish', 'sexual', 'America', 'own', 'family', 'poem', 'George', 'call', 'poetry', 'John', 'brother', 'University', 'part', 'would']
Topic: 9 
Words: ['Whitman', 'Walt', 'write', 'Grass', 'work', 'poet', 'New', 'Leaves', 'leave', 'York', 'poetry', 'call', 'one', 'american', 'edition', 'time', 'America', 'publish', 'other', 'poem', 'own', 'first', 'would', 'part', 'family', 'include', 'know', 'life', 'move', 'brother']


In [320]:
#print topics and words
poet1_history=set()
for index, topic in lda_modelpoet1.show_topics(formatted=False, num_words= 30):
  for x in topic:
    poet1_history.add(x[0])

print('Poet1 important Topic from History/Wiki \n')
print(poet1_history)

Poet1 important Topic from History/Wiki 

{'brother', 'edition', 'include', 'Walt', 'life', 'George', 'move', 'publish', 'other', 'Grass', 'family', 'would', 'own', 'poet', 'sexual', 'year', 'poem', 'poetry', 'know', 'America', 'write', 'Whitman', 'time', 'american', 'call', 'University', 'part', 'leave', 'Leaves', 'York', 'John', 'first', 'work', 'New', 'one'}


**POET2**

In [274]:
final_corpus2=[]
id2wordd2 = corpora.Dictionary([poet_2_lemma_corpus])
for text in [poet_2_lemma_corpus]:
    new = id2wordd2.doc2bow(text)
    final_corpus2.append(new)

In [286]:
lda_modelpoet2 = gensim.models.LdaMulticore(final_corpus2, num_topics=2, id2word=id2wordd2, passes=2, workers=2)

In [287]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_modelpoet2, final_corpus2, id2wordd2)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.003974  0.0       1        1  99.826441
0     -0.003974  0.0       2        1   0.173559, topic_info=        Term       Freq      Total Category  logprob  loglift
49      Dyer  10.000000  10.000000  Default  30.0000  30.0000
51    Edward   7.000000   7.000000  Default  29.0000  29.0000
127      Sir   4.000000   4.000000  Default  28.0000  28.0000
89       May   3.000000   3.000000  Default  27.0000  27.0000
252        s   3.000000   3.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
240   public   0.002508   1.770304   Topic2  -5.5207  -0.2030
181   domain   0.002473   1.778245   Topic2  -5.5349  -0.2216
274    think   0.002463   1.780499   Topic2  -5.5390  -0.2270
38   College   0.002453   1.782702   Topic2  -5.5429  -0.2322
12      1607   0.002576   2.532916   Topic2  -5.4938  -0.5343

[120 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
6         1  1.149204     1589
10        1  0.936726     1600
12        1  1.184406     1607
15        1  0.944246     1888
32        1  0.942758   Brooks
...     ...       ...      ...
270       1  0.940280  survive
274       1  1.123280    think
279       1  0.939777     west
281       1  0.942408     work
283       1  1.114620    write

[69 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [293]:
for index, topic in lda_modelpoet2.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

Topic: 9 
Words: ['Dyer', 'Edward', 'Sir', 'May', 's', 'work', 'Elizabeth', 'poem', 'court', 'Somerset', 'Oxford', 'one', 'Phillis', 'Steven', 'Earl', '1589', 'queen', 'English', 'William', 'Wood', '1607', 'Woodstock', 'make', 'poet', 'say', 'Sidney', 'public', 'domain', 'think', 'College']
Topic: 9 
Words: ['Dyer', 'Edward', 'Sir', 'May', 'work', 's', '1607', 'Oxford', 'Somerset', 'court', 'poem', 'Elizabeth', 'book', 'Grosart', 'grant', 'Thomas', 'Poems', 'Works', 'seem', 'write', 'Life', 'College', 'think', 'domain', 'public', 'Sidney', 'say', 'poet', 'make', 'Woodstock']


In [314]:
#print topics and words
poet2_history=set()
for index, topic in lda_modelpoet2.show_topics(formatted=False, num_words= 30):
  for x in topic:
    poet2_history.add(x[0])

print('Poet2 important Topic from History/Wiki \n')
print(poet2_history)

Poet1 important Topic from History/Wiki 

{'Poems', 'Somerset', 'Oxford', 'Elizabeth', 'seem', 'grant', 'Phillis', 'court', 'queen', '1607', 'College', 'Thomas', 'English', 's', 'May', 'poem', 'Woodstock', 'make', 'poet', 'domain', 'Wood', 'Sir', 'say', 'public', 'write', 'Earl', 'Edward', 'book', 'think', 'Grosart', '1589', 'Dyer', 'Sidney', 'Life', 'Steven', 'Works', 'William', 'work', 'one'}
